# Passwort Generator
Das Projekt hat sich die Erstellung eines Passwort-Generators als Ziel gesetzt. Dabei sollen die zu erstellenden Passwörter möglichst denen eines Menschen ähneln. 
Zur Realisierung wurde die RNN- Architektur verwendet. Diese Recurrent Neural Networks ermöglichen es, Voraussagen mittels eines Kontext zu treffen, der durch frühere Inputs entstanden ist. Das Netzwerk verfügt sozusagen über ein Gedächtnis. In der Umsetzung geschieht dies durch die Kombination des Hidden-Layern aus der vorherigen Sequenz mit den Hidden-Layern aus der aktuellen Sequenz. Die vorherigen Hidden-Layer haben damit Einfluß auf den Output der nächsten Sequenz. Dieser Algorithmus wird in einer Schleife abgebildet, bis sämtliche Inputs verarbeitet wurden und der Kontext ersichtlich ist.
Klarer wird dies mit den nachfolgenden Formeln, mit denen das Netzwerk trainiert wird:

\begin{align}
\ h_t  = f(W^{hh}h_t-1 + W^{hx} + x_t \\
\ y_t  = softmax(W^Sh_t) \\
\ J^t(\theta)  =\sum_{i=1}^{[V]} (y_{ti}\log(y_{ti}))
\end{align}

Die erste Formel ist dafür da, sich an die Hidden-Layer des vorherigen Durchlaufs zu "erinnern". Dabei wird durch h-1 auf den vorherigen Hidden-Layer zugegriffen. Dies wird kombiniert mit dem aktuellen x, auch wird anschließend eine Akivierungsfunktion durchgeführt, am gebräuchlisten sind hierbei der Tangens hyperbolicus oder die Sigmoid-Funktion.
Die zweite Formel kümmert sich um die Voraussage des nächsten Ergebnisses in Form von einer Wahrscheinlichkeitsverteilung. 
Zum Schluss wird in der dritten Formel mittels der Cross-Entropy-Loss-Funktion der Fehler zwischen dem Input und dem Output berechnet.

/EOS noch erklären, / Quellen angeben / Quellcode mehr kommentieren

In [4]:
#Imports
from __future__ import unicode_literals, print_function, division
from io import open
import wget # to download passwordlist
import glob
import os
import random
import numpy as np
import unicodedata
import string
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Auslesen der Passworddatei:

Beim Einlesen der Daten haben wir uns an das Tutorial von Pytorch im Kontext mit RNNs gehalten. Um die Strings verarbeiten zu können, haben wir die Methode aus dem Tutorial übernommen, die die Zeichen in Ascii-Code umwandelt, um eine numerische Darstellung der Passwörter zu erhalten. Weiterhin erfolgt eine Umwandlung zu Tensoren sowohl für die Passwörter als auch für die Targets, damit die Backpropagation angewendet werden kann, um den Loss zu berechnen.

In [5]:
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in letters
    )

def readPasswords(filename):
    passwords = []
    with open(filename, 'r', encoding="utf8", errors='ignore') as f:
        for line in f:
            if len(line) > 1:
                passwords.append(line)
    passw = [unicodeToAscii(password) for password in passwords]
    return passw

def charToIndex(char):
    return letters.find(char)

def passwordToTensor(name):
    ret = torch.zeros(len(name), 1, len_letters)
    for i, char in enumerate(name):
        ret[i][0][charToIndex(char)] = 1
    return ret

def targetToTensor(password):
    indizes = [letters.find(password[i]) for i in range(1,len(password))]
    indizes.append(len_letters - 1) #EOS
    return torch.LongTensor(indizes)
                                                        
    
letters = string.ascii_letters + string.digits + string.punctuation
len_letters = len(letters) + 1

## Einlesen der Passwortlisten

Um unser Projekt möglichst leicht reproduzierbar zu machen, haben wir eine Funktion eingebaut, die automatisch Passwortlisten runterlädt, falls diese noch nicht vorhanden sind. So ist gewährleistet, dass Interessenten, die selbst das Netzwerk trainieren möchten, nicht erst umständlich Passwortlisten runterladen müssen. Die Passwortlisten wurden vorher durch ein Skript von Passwörtern gereinigt, die Zeichen enthielten, die nicht UTF-8 -kompatibel waren, da diese Passwörter beim Einlesen Fehler erzeugt haben. Zusammen kommen wir auf ungefähr 83 Millionen Passwörter, die wir das Netzwerk verwenden können.

In [6]:
urls = ['https://www.scrapmaker.com/data/wordlists/dictionaries/rockyou.txt',
        'https://www.scrapmaker.com/download/data/wordlists/passwords/thelist.txt']
filelist = []
passwords = []

# exist files
dirs = os.listdir()
for file in dirs:
    if file.endswith(".txt"):
        filelist.append(file)

for url in urls:
    file = url.split("/")[-1]
    #download files if not exists
    if file not in filelist:
        wget.download(url)
        print('\n successful downloaded ', url)
    #read file and append to passwordlist
    passwords += readPasswords(file)
    print('successful read', file)
    print('total passswords in list: ', len(passwords))

#entfernen von leeren Zeilen
passwords = [passw for passw in passwords if passw != '']

successful read rockyou.txt
total passswords in list:  1472
successful read passwords-20MB.txt
total passswords in list:  1527
successful read cracklib.txt
total passswords in list:  1582


### Aktivierungsfunktionen
    nn.LogSoftmax() - 
    nn.LeakyReLU() - 
    nn.LogSigmoid() -
    nn.Tanh() - 
## Generatorklasse:

In der Generatorklasse werden die grundlegenden Funktionen und Variablen festgelegt, mit denen das RNN initialisert und ausgeführt werden kann. Der Aufbau ist auch hier stark an die Vorlage aus dem Pytorch-Grundlage angelehnt, da diese leicht verständlich und dem theoretischen Prinzip eines RNN am ehesten entsprach. Allerdings haben wir, wie bereits in unserem Exposé beschrieben, andere Aktivierungsfunktionen eingefügt, die beliebig ausgetauscht werden können, um die, je nach Aktivierungsfunktion, entstandenen Ergebnisse vergleichen zu können. Auf diesem Wege können die Auswirkungen der verschiedenen Aktivierungsfunktionen besser begutachtet werden. Auch haben wir uns an den letzten Vorlesungen orientiert und ein Dropout eingefügt, welcher besagt, wie hoch der Prozentsatz der inaktiven Neuronen pro Durchlauf sein soll. Auch hier bietet sich ein Verändern des Parameters an, um die Auswirkungen an den erstellten Passwörtern des Neuronalen Netzes zu beobachten

In [ ]:
#https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
class PasswordGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PasswordGenerator, self).__init__()
        self.hidden = hidden_size
        self.input2hidden = nn.Linear(input_size + hidden_size, hidden_size, bias=True)
        self.input2output = nn.Linear(input_size + hidden_size, output_size, bias=True)
        self.output2output = nn.Linear(hidden_size +  output_size, output_size, bias=True)        
        self.dropout = nn.Dropout(0.3)
        self.softmax = nn.LogSoftmax(dim=1)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.LogSigmoid()
        self.tanh = nn.Tanh()
        
        
    def forward(self, input, hidden):
        combined_input = torch.cat((input, hidden), dim=1)
        hidden = self.input2hidden(combined_input)
        output = self.input2output(combined_input)
        combined_output = torch.cat((hidden, output), dim=1)
        output = self. output2output(combined_output)        
        output = self.dropout(output)
        #output = self.softmax(output)
        
        #output = self.sigmoid(output)
        output = self.relu(output)
        #output = self.tanh(output)
        
        return hidden, output

    def initHidden(self):
        return torch.zeros(1, self.hidden)

### Loss Funktionen
    nn.BCELoss() - Binary Cross Entropy
    nn.BCEWithLogitsLoss() - This loss combines a Sigmoid layer and the BCELoss in one single class
    nn.NLLLoss() - negative log likelihood loss
    nn.CrossEntropyLoss() - combines nn.LogSoftmax() and nn.NLLLoss() in one single class
    
## Training:

Beim Training definieren wir zunächst unser Model, in dem wir den Inputgröße, die Anzahl der Hidden Layers und die Größe des Outputs angeben.
Durch die Bindung der Loss-Function an einen Parameter lassen sich auch hier bequem andere Loss-Functions testen, um die Auswirkungen dieser auf den Trainingsprozess zu beobachten.
/ Was macht unsqueeze mit -1?
Aus Tutorials haben wir erfahren, dass die Methode zero_grad() unbedingt in der Trainingsmethode vorhanden sein muss, um den berechneten Fehler nicht zu addieren. Damit wird sichergestellt, dass der Fehler wieder zurückgesetzt wird und neue Werte annehmen kann.
In einer Schleife wird dann der Forward-Pass und die Berechnung des Loss ausgeführt. Nach der Schleife kann der berechnete Fehler zurückgerechnet werden, um die Hyperparameter anzupassen.
Ebenfalls haben wir eine kleine Funktion aus dem Pytorch-Tutorial übernommen, um Plot-Daten aus dem Model zu extrahieren.
In der übergeordneten train() - Methode nehmen wir uns zufällige Passwörter aus der Liste, um mit diesen unser Netz zu trainieren. Diese ausgewählten Passwörter werden dann, wie eingangs erwähnt, zu Tensoren umgewandelt, damit die entsprechenden Methoden zum Training anwendbar sind. Diese Tensoren werden nun trainPasswords() übergeben, um anschließend den Gesamtfehler zu erhalten.

//Abschnitt mit progress und c erklären

In [ ]:
model = PasswordGenerator(len_letters, len_letters, len_letters)
loss_fn = nn.CrossEntropyLoss() #define Loss Function
learning_rate = 0.0005

def trainPasswords(input, target):
    target.unsqueeze_(-1)
    hidden = model.initHidden()
    model.zero_grad() # zeroes the gradient buffers of all parameters
    loss = 0
    for i in range(input.size()[0]):
        hidden, output = model(input[i], hidden)
        l = loss_fn(output, target[i]) # Compute the loss
        loss += l
    loss.requires_grad_(True) # The autograd package provides automatic differentiation for all operations on Tensors
    loss.backward()
    for p in model.parameters():
        p.data = p.data.add(-learning_rate, p.grad.data)
        
    return output, loss.item() / input.size(0)

def train(trainrounds):
    total_loss = 0
    plots = []
    plot_every = 100
    progress = 0
    c = 0
    
    for j in range(0, trainrounds):
    #for j in range(len(passwords)):
            #password = passwords[j]
            password = random.choice(passwords)
            #print('picked password:', password)
            input = passwordToTensor(password)
            target = targetToTensor(password)
            output, loss = trainPasswords(input, target)
            total_loss += loss
            
            progress = j / trainrounds * 100
            if (c < round(progress) and round(progress) % 5 == 0) or j == 1:
                c = round(progress)
                print(round(progress), '% made. Loss: ', loss)
            if j % plot_every == 0:
                plots.append(total_loss / plot_every)
                total_loss = 0

    plt.figure()
    plt.plot(plots)

## Sample Password:

In diesem Bereich angekommen, haben wir bereits das Netz vollständig trainiere und können uns Passwörter generieren lassen.
Da, wie Anfangs erklärt, die Hidden Layer der vorherigen Sequenz Einfluß nehmen auf die Hidden Layer der aktuellen Sequenz, bedeutet dies gleichzeitig, dass beim ersten Durchgang kein vorheriger Hidden Layer existiert. Daher geben wir den ersten Input vor, um dessen anschließende Hidden Layers an die nächste Sequenz zu übergeben.

/ Rest aus der Schleife noch erklären

In [ ]:
max_chars = 10 # max 10 chars for password

def sample(start_letter='a'):
    with torch.no_grad():  # no need to track history in sampling
        input = passwordToTensor(start_letter)
        hidden = model.initHidden()

        output_name = start_letter

        for i in range(max_chars):
            output, hidden = model(input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == len_letters - 1:
                break
            else:
                letter = letters[topi]
                output_name += letter
            input = passwordToTensor(letter)

        return output_name
#before train
random_start_char = random.choice(letters)
print('Sampled Password: ', sample(random_start_char))

#train
#train(range(len(passwords)))
train(100000)

#after train
print('Sampled Password: ', sample(random_start_char))
print('Sampled Password: ', sample(random_start_char))
print('Sampled Password: ', sample(random_start_char))
print('Sampled Password: ', sample(random_start_char))

Sampled Password:  rGYr;.t.t.t
0 % made. Loss:  4.552676816529866
5 % made. Loss:  nan
